In [ ]:
# read stl file to get coords and interleave
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot

# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('cube.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

# Auto scale to the mesh size
scale = your_mesh.points.flatten(-1)
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()

In [17]:
# read gcode to get coords and interleave
# need dimensions of a slice
# 10 x 10 x 10 mm for many_layers_cube.gcode
import json
import os
import numpy as np
from PIL import Image
from PIL import ImageOps
from skimage import color as skcolor
from skimage import io as skio
from skimage import novice as sknov
from skimage import draw as skdr
import matplotlib.pyplot as plt
from scipy.misc import imresize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection

### CONSTANTS

MATERIAL_1 = "material 1"
MATERIAL_2 = "material 2"
MATERIAL_3 = "material 3"

# noop material
MATERIAL_NOOP = "noop"

# color map for visualizing print
COLOR_MAP = {MATERIAL_1: 'r', MATERIAL_2: 'g', MATERIAL_3: 'b'}

###

class GCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, x, y, z, material, usecs=100):
        """
        Init
        x, x location in gcode coords
        y, y location in gcode coords
        z, z location in gcode coords
        material, material indicator
        usecs, delay after movement
        """
        self.x = x
        self.y = y
        self.z = z
        self.material = material
        self.usecs = usecs
    
    def __str__(self):
        """Returns gcode representation of command"""
        if self.material == MATERIAL_1:
            return "T0; G0 E10; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_2:
            return "T1; G0 E20; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_3:
            return "T2; G0 E30; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_NOOP:
            return ""

        raise ValueError("Unknown material: {}".format(self.material))

def convert_to_gcode(width, height, start_x, start_y, start_z, z_unit, num_layers=1, usecs=600, grid_unit=0.5, flip_flop=True):
    """
    Convert a list of binary images to gcommands. Iterates over each pixel
    and forms a GCommand object

    binary_layers, list of ndarrays of dimension height x width (elements are 0 or 1)
    usecs, delay for GCommand
    grid_unit, conversion of pixel to gcode dimensions in x and y
    z_unit, conversion of pixel to gcode dimensions in z
    start_x, start x location in gcode
    start_y, start y location in gcode
    flip_flop, boolean flip scans of left and right to minimize tracking

    return gcommand_layers, list of lists of GCommand objects
    """
    gcommand_layers = []

    for grid_z in range(num_layers):
        gcommands = []

        for grid_y in range(height):
            if grid_y % 2 == 0 and flip_flop:
                x_iterator = reversed(range(width))
            else:
                x_iterator = range(width)
            for grid_x in x_iterator:
                #pixel = binary_layers[grid_z][grid_y, grid_x]
                material = "material 1" # temporary

                if material is not MATERIAL_NOOP:
                    gcommand = GCommand(grid_x * grid_unit + start_x, \
                                        grid_y * grid_unit + start_y, \
                                        grid_z * z_unit + start_z, \
                                        material, \
                                        usecs)
                    gcommands.append(gcommand)

        gcommand_layers.append(gcommands)

    return gcommand_layers


def write_gcode(layered_content, gcommand_layers, gcode_path, layer_names=None, heatbed_temp=37):
    """
    Convert list of gcommands into .gcode file. Also add start and end commands

    gcommand_layers, list of lists of GCommand objects
    gcode_path, path to write output
    layer_names, names for each layer, defaults to one index naming
    heatbed_temp, start temp UNUSED
    """
    assert heatbed_temp <= 200, "{} > max temp 200".format(heatbed_temp)

    if layer_names is not None:
        assert len(layer_names) == len(gcommand_layers), \
                "not enough names for all layers, remove layer_names to default naming"

    with open(gcode_path, 'w') as gcode_file:
        gcode_file.write(layered_content[0])
        
        for layer_index, gcommands in enumerate(gcommand_layers):
            for gcommand in gcommands:
                gcode_file.write(str(gcommand))
            gcode_file.write(layered_content[layer_index+1])

def main():
    with open('many_layers_cube.gcode') as f:
        content = f.readlines()
    layered_content = []
    layer, ctr = "", 0
    for c in content:
        if 'G1 Z' in c and 'G1 Z5' not in c:
            layer += ";layer: " + str(ctr) + "\n"
            layered_content.append(layer)
            layer, ctr = "", ctr+1
        layer += c
        
    layer += ";layer: " + str(ctr) + "\n"
    layered_content.append(layer)
        
    for l in layered_content:
        print l
    # find slice dimensions: I think this is better as a configuration
    # X89.915 Y90.753
    # slice dimensions:
    # width, height = 10 # because gcode units are mm
    # each dot is .3 mm apart
    width, height = 34, 34
    start_x, start_y = 90, 90
    
    # find start_z and z_unit. could this be a config?
    # hardcoding for now
    start_z, z_unit = 0.350, .3
    gcommand_layers = convert_to_gcode(width, height, start_x, start_y, start_z, z_unit, len(layered_content)-1)
    
    write_gcode(layered_content, gcommand_layers, 'test.gcode')

if __name__ == '__main__':
    main()


; generated by Slic3r 1.2.9 on 2017-04-15 at 17:10:19

; external perimeters extrusion width = 0.50mm
; perimeters extrusion width = 0.72mm
; infill extrusion width = 0.72mm
; solid infill extrusion width = 0.72mm
; top infill extrusion width = 0.72mm

M107
M104 S200 ; set temperature
G28 ; home all axes
G1 Z5 F5000 ; lift nozzle

M109 S200 ; wait for temperature to be reached
G21 ; set units to millimeters
G90 ; use absolute coordinates
M82 ; use absolute distances for extrusion
G92 E0
;layer: 0

G1 Z0.350 F7800.000
G1 E-2.00000 F2400.00000
G92 E0
G1 X89.915 Y90.753 F7800.000
G1 E2.00000 F2400.00000
G1 X91.668 Y89.274 E2.07096 F1800.000
G1 X93.820 Y88.481 E2.14192
G1 X95.000 Y88.375 E2.17858
G1 X105.000 Y88.375 E2.48799
G1 X107.259 Y88.772 E2.55895
G1 X109.247 Y89.915 E2.62991 F1800.000
G1 X110.726 Y91.668 E2.70088
G1 X111.519 Y93.820 E2.77184
G1 X111.625 Y95.000 E2.80849
G1 X111.625 Y105.000 E3.11790
G1 X111.228 Y107.259 E3.18886
G1 X110.085 Y109.247 E3.25983
G1 X108.332 Y110.726 E3.